<a href="https://colab.research.google.com/github/duyquoc1508/Bag_Of_Words/blob/master/source/Social_network_analytis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Phân tích và trực quan hóa dữ liệu mạng xã hội**
### Mạng xã hội: *Facebook*
### Group: *UIT K11*
https://www.facebook.com/groups/UIT.K2016

## Thu thập dữ liệu và tiền xử lí

Đọc tất cả các user trong group UIT K11

In [0]:
import pandas as pd 
data = pd.read_csv("UIT.K11.csv") 
data.head()

In [0]:
list_uid_members = data["UID"].tolist()
print(list_uid_members)

In [0]:
list_uid_members_str = list(map(str,list_uid_members))
print(list_uid_members_str)

Gọi tới api của Facebook để lấy bạn bè của tất cả user

In [0]:
import requests as req
token = 'EAAAAZAw4FxQIBAD8OB614aWaBpV9Jc3JjtXegHUasc0UX4ZA18Vk065sewsxSZA7LVoYZAPvnAIlHZAuI3SX8XNze3qZAEOOaT1NBqiK1yusYUk0yZA3QBKOHqHAf0ZCmwwGZBQF6KJ9VH9pE09iRMJZCP3Q5Qszrsjx39ZAvczngAMuQZDZD'

Lấy ra tất cả các user trong group đặt chế độ public friend trên Facebook. Các user không public friend coi như không nằm trong group

In [0]:
list_user_public_friend = []
for uid in list_uid_members_str:
  try:
    res = req.get('https://graph.facebook.com/v4.0/'+ uid + '/friends?fields=id', params={'access_token': token})
    friends_of_user = res.json()['data']
    # Nếu friends_of_user != null => user này đặt chế độ public friend 
    if friends_of_user:
      list_user_public_friend.append(uid)
  except:
    continue

In [0]:
print(list_user_public_friend)

In [0]:
print(len(list_user_public_friend))

Định dạnh trả về của api

In [0]:
res = req.get('https://graph.facebook.com/v4.0/100010075866425/friends?fields=id', params={'access_token': token})
print(res.json())

Với mỗi user chỉ chọn ra các bạn bè nằm trong group UIT K11. Đồng thời ghi vào file

In [0]:
import csv
with open('destination.csv', mode='w+') as csvFile:
  writer = csv.writer(csvFile, delimiter=',')

  for uid in list_user_public_friend:
    res = req.get('https://graph.facebook.com/v4.0/'+ uid + '/friends?fields=id&&limit=5000', params={'access_token': token})
    friends_of_user = res.json()['data']
    friends_in_group = []
    for friend in friends_of_user:
      # Chỉ chọn ra những bạn bè nằm trong danh sách public friend
      if friend['id'] in list_user_public_friend:
        friends_in_group.append(friend['id'])
    row = [uid, *friends_in_group]
    writer.writerow(row)

csvFile.close()

## Tính toán các độ đo trong mạng

In [0]:
f = open("destination.csv", "r")
data= f.read()
file = open('destination.csv', 'r+');
data = file.read().replace(',,','').replace(',\n','\n')

# Loại bỏ đi phần tử cuối vì dòng cuối cũng kết thúc bằng kí hiệu \n
rows = data.split('\n')[:-1]

Chuyển thành mảng 2 chiều với mỗi phần tử trong mảng là 1 dòng trong dữ liệu

In [0]:
def convert_to_2D_array(rows):
    return [row.split(',') for row in rows]

In [0]:
array_2D = convert_to_2D_array(rows)
print(array_2D)

In [0]:
import networkx as nx
g = nx.Graph()

# Lấy phần tử [i][0] truyền vào làm nodes
for node in array_2D:
  g.add_node(node[0])

# Thêm cạnh cho đồ thị bằng cách bắt cặp phần tửi [i][0] với [i][j]
for item in array_2D:
  for index in range(1,len(item)):
    g.add_edge(item[0], item[index])

In [0]:
print(g.nodes())

In [0]:
print(g.edges())

Đọc file các uid và name thành dictionary để hiển thị tên lúc tính toán

In [0]:
reader = csv.reader(open('UIT.K11.csv', 'r'))
dict_user_uid = dict()

for row in reader:
  name, uid = row
  dict_user_uid[uid] = name
# Remove header
del dict_user_uid['UID']
print(dict_user_uid)

In [0]:
from prettytable import PrettyTable
# Hiển thị uid, tên và kết quả của độ đo
def findUser(dict_top10, dict_user_uid):
  t = PrettyTable(['UID','Name','Centrality'])
  for item in dict_top10:
    t.add_row([item, dict_user_uid[item], dict_top10[item]])
  print(t)

# Chọn ra 10 user có độ đo lớn nhất
def dict_top10(dict_centrality):
  arr_top10 = sorted(dict_centrality.items(), key=lambda kv: kv[1], reverse=True)[:10]
  dict_top10 = dict()
  for index, value in arr_top10:
    dict_top10[index] = value
  return dict_top10

**Degree Centrality**

In [30]:
dict_degree_centrality = dict( nx.degree_centrality(g))

dict_top10_degree = dict_top10(dict_degree_centrality)
print('Top 10 Key Player - Degree Centrality')
findUser(dict_top10_degree, dict_user_uid)

Top 10 Key Player - Degree Centrality
+-----------------+-----------------+---------------------+
|       UID       |       Name      |      Centrality     |
+-----------------+-----------------+---------------------+
| 100022371771809 |     Toàn Lê     | 0.18611111111111112 |
| 100027261426135 |     Bảo Phúc    | 0.17222222222222222 |
| 100012780131803 | Dương Đức Thành | 0.13194444444444445 |
| 100003782937571 |    Phương Lan   | 0.11875000000000001 |
| 100005101389366 |   Cường Nguyễn  | 0.10833333333333334 |
| 100007858844667 |      Võ Tân     | 0.10277777777777779 |
| 100028418463108 | Nguyễn Minh Kha | 0.10208333333333333 |
| 100005704122021 |  Nguyễn Thị Tâm | 0.10138888888888889 |
| 100009961336237 |  Nguyễn Hữu Thọ | 0.09791666666666667 |
| 100013232892681 |   Nguyễn Minh   | 0.09027777777777778 |
+-----------------+-----------------+---------------------+


**Betweenness Centrality**

In [0]:
dict_betweenness_centrality = dict( nx.betweenness_centrality(g))

dict_top10_betweenness = dict_top10(dict_betweenness_centrality)
print('Top 10 Key Player - Betweenness Centrality')
findUser(dict_top10_betweenness, dict_user_uid)

**Closeness Centrality**

In [0]:
dict_closeness_centrality = dict( nx.closeness_centrality(g))

dict_top10_closeness = dict_top10(dict_closeness_centrality)
print('Top 10 Key Player - Centrality Centrality')
findUser(dict_top10_closeness,dict_user_uid)

**Eigenvector Centrality**

In [0]:
dict_eigenvector_centrality = dict( nx.eigenvector_centrality(g))

dict_top10_eigenvector = dict_top10(dict_eigenvector_centrality)
print('Top 10 Key Player - Eigenvector Centrality')
findUser(dict_top10_eigenvector,dict_user_uid)

## Trực quan hóa dữ liệu

In [0]:
!pip install python-igraph

Danh sách tất cả các node trong đồ thị

In [0]:
nodes = [item[0] for item in array_2D]
print(nodes)

In [0]:
Tạo danh sách cạnh có dạnh index của các node
Ex: ('100010630570591','100003265219762') -> (0,819)

In [0]:
edges = []
for i in range(len(array_2D)):
  for item in array_2D[i][1:]:
    edges.append((i, nodes.index(item)))
print(edges)

In [0]:
number_node = len(nodes)
print(numbers_node)

In [0]:
number_edge = len(edges)
print(number_edge)

Tạo đồ thị

In [0]:
import igraph as ig
G=ig.Graph(edges, directed=False)

Tạo vị trí cho các nodes chế độ Kamada-Kawai 3 chiều

In [0]:
layt=G.layout('kk', dim=3)

In [0]:
print(type(layt))
print(layt[1])
print(layt)

Chuyển tọa độ từ [(x1,y1,z1),(x2,y2,z2),...] => X = [x1,x2,...] , Y = [...]

In [0]:
# Chuyển tọa độ nodes
Xn=[layt[k][0] for k in range(number_node)]# tọa độ theo trục x của node
Yn=[layt[k][1] for k in range(number_node)]# tọa độ theo trục y
Zn=[layt[k][2] for k in range(number_node)]# tọa độ theo trục z
# Chuyển tọa độ cạnh
Xe=[]
Ye=[]
Ze=[]
for e in edges:
  Xe+=[layt[e[0]][0],layt[e[1]][0], None]# tọa độ theo trục x của đầu cạnh
  Ye+=[layt[e[0]][1],layt[e[1]][1], None]
  Ze+=[layt[e[0]][2],layt[e[1]][2], None]

Giải thích code bên trong vòng lặp ở trên

In [0]:
e = edges[0]
print('e: ',e)
print('layt[e[0]]: ',layt[e[0]])
print('layt[e[0]][0]: ',layt[e[0]][0])
print('layt[e[1]]: ',layt[e[1]])
print('layt[e[1]][0]: ',layt[e[1]][0])

In [0]:
print(Xn)

In [0]:
print(Xe)

Chuyển qua tên để vẽ đồ thị

In [0]:
info = []
for i in range(len(nodes)):
  uid = nodes[i]
  name = dict_user_uid[uid]
  info.append(nodes[i]+'<br>'+name)
print(info)

In [0]:
print(nodes)
print(info)

In [0]:
from plotly.offline import plot
import plotly.graph_objs as go
#Thêm cạnh
trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
#Thêm node
trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               marker=dict(symbol='circle',
                             size=5,
                             color='rgb(51, 190, 214)',
                             line=dict(color='rgb(50,50,50)', width=0.3),
                             ),
               text=info,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Network of friends in the group UIT K11 (3D visualization)",
         width=1024,
         height=768,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
    hovermode= 'closest'
         )

In [0]:
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)
plot(fig, filename='3d_network_graphs.html')